**Script to map two lists of person names**

This notebook contains the steps for mapping two lists of person names (ListA and ListB) and gives as a result a list of possible candidates with scores.

This script is written by Liliana Melgar-Estrada for the SKILLNET PROJECT (https://skillnet.nl/)

Last update: June 17, 2022

# Data preparation (externally, before importing)

The script works with two lists of person names which contain basic colums: 
- personId: this can be any Id used in List A or List B
- Name string: this is the name of the person without any dates, roles, or alternative names in it
- Date of birth: in the form YYYY is the year in which the person was born. If uncertain, don't add any marks here, that can be done in a separate column
- Date of death: same as with the date of birth 
- Date of flourish (floriat or fl.): also in the form YYYY, it is the year in which a person is known to be active or alive
- Any other addidional columns useful for dissambiguation can be added, e.g., occupations, but these have to be added manually to the script below.

Important to take into account:
- Any notes or clarifications about dates or names, uncertainty notes, alternative names, roles, etc., should be added in a separated column(s).
- If a person has both date of birth and date of death, it is NOT necessary to add a floriat date, if added, the names that have them will be ignored in the script below!

If the person names are part of a file that is in the "letter" format (i.e., senders are in one column and receivers are in another column) use the other notebook: "Wp02_MappingPersonNames2_LetterFormat_WithCorrespondents.ipynb".

# Import libraries

In [ ]:
import matplotlib
import pandas as pd
import numpy as np
import re
import fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
# import jellyfish

from IPython.display import display
from IPython.display import clear_output

import csv

from IPython.display import display, HTML
# display(HTML("<style>.container { width:95% !important; }</style>"))
# pd.options.display.max_columns = 10
pd.options.display.max_rows = 1000
# pd.options.display.width = 1000

# to add timestamp to file names
import time

# for progress bar (https://datascientyst.com/progress-bars-pandas-python-tqdm/)
from tqdm import tqdm
from time import sleep

# Import files

In [ ]:
# Test data is located in the repository folder indicated in the path here
# this is the local path to the raw data in your own computer to where you downloaded/cloned the repository
pathRawDataFolder = f'/Users/Melga001/stack/workspace/SKILLNET-PRODUCTION/_sharedRepositoriesGithub/mappingPersonNames/data/raw/'

## Import ListA

For the test version, ListA contains unique names from the Catalogus Epistolarum Neerlandicarum (CEN) extracted from a slice of correspondents from van Leeuwenhoek and Swammerdam (internal note: cy08).

In [ ]:
# Import here the first file (ListA), this is the names you want to map the other list to.
# the list is imported as a pandas dataframe
dfA_t0 = pd.read_csv(f"{pathRawDataFolder}ListA.csv", sep = ",", index_col=False, engine='python')

In [ ]:
dfA_t0.info()

## Import list to map to (LIST B)

For the test version, ListB contains unique names from the Epistolarium (http://ckcc.huygens.knaw.nl/epistolarium/)  (internal note: cy13).

In [ ]:
# Import here the second file (ListB), these are the names you want to map (find a match) to the initial list.
# the list is imported as a pandas dataframe
dfB_t0 = pd.read_csv(f"{pathRawDataFolder}ListB.csv", sep = ",", index_col=False, engine='python')

In [ ]:
dfB_t0.info()

# Prepare ListA and ListB

In this step the data is prepared for the mappings (reassigning column names and changing data types in case they were not the right ones)

## Prepare ListA

In [ ]:
# assign column names
dfA_t0.columns = ['personIdA',
                   'nameStringA',
                   'dateBirthA', 
                   'dateDeathA', 
                   'dateFlA'
                   ]

In [ ]:
# make a copy of the dataframe and rename it
dfA = dfA_t0.reset_index(drop=True)

In [ ]:
# convert datatypes and fill in empty values
dfA_columns = dfA.columns
for column in dfA_columns:
    dataType = dfA.dtypes[column]
    if dataType == np.float64:
        dfA[column] = dfA[column].fillna(0.0)
        dfA[column] = dfA[column].astype(int)
    if dataType == object:
        dfA[column] = dfA[column].fillna('null')
        dfA[column] = dfA[column].astype(str)

In [ ]:
dfA.info()

In [ ]:
dfA.head(10)

## Prepare ListB

In [ ]:
# assign column names
dfB_t0.columns = [
                   'personIdB',
                   'nameStringB', 
                   'dateBirthB', 
                   'dateDeathB', 
                   'dateFlB',
                   ]

In [ ]:
# make a copy of the dataframe and rename it
dfB = dfB_t0.reset_index(drop=True).copy()

In [ ]:
# convert datatypes and fill in empty values
dfB_columns = dfB.columns
for column in dfB_columns:
    dataType = dfB.dtypes[column]
    if dataType == np.float64:
        dfB[column] = dfB[column].fillna(0.0)
        dfB[column] = dfB[column].astype(int)
    if dataType == object:
        dfB[column] = dfB[column].fillna('null')
        dfB[column] = dfB[column].astype(str)

In [ ]:
dfB.info()

In [ ]:
dfB.head(10)

## Store listA and listB for future reference

In [ ]:
# this is the local path to the temporary data in your own computer to where you downloaded/cloned the repository
pathTempDataFolder = f'/Users/Melga001/stack/workspace/SKILLNET-PRODUCTION/_sharedRepositoriesGithub/mappingPersonNames/data/temp/'

In [ ]:
# this inserts the timestamp in the file name
timestr = time.strftime("%Y%m%d-%H%M%S")

fileListA = (f"{pathTempDataFolder}ListA_{timestr}.csv")
dfA.to_csv(fileListA)

fileListB = (f"{pathTempDataFolder}ListB_{timestr}.csv")
dfB.to_csv(fileListB)

## Create a dataframe to store the mappings

In [ ]:
dfC = pd.DataFrame()

# Run mapping script

Here below there is the mapping script that will compare the names in listB with the names in listA checkign if the name string matches and, if so, it applies some rules to determine if the respective dates of birth/death/fl. have a logical relation. If so, a mapping candidate is added to the dataframe C.

This script is also stored separately here: 

The counter shows:
|percentage done|items processed/total items \[time passed < time left, number of iterations per second\]

In [ ]:
##### PASTE HERE THE SCRIPT AVAILABLE IN THIS PATH: 
### {your path to repository}/mappingPersonNames/src/personMappingScript-v44-20220620.py

In [ ]:
dfC.info()

In [ ]:
dfC.head(5)

In [ ]:
# test = dfC[dfC.scoreCase.str.contains('L-')]

In [ ]:
# test

In [ ]:
dfC.scoreCase.value_counts()

# Prepare mapping output for analysis

#### Replace the .0 in person dates and convert to strings

In [ ]:

dfC['dateBirthA'] = dfC['dateBirthA'].astype(str).replace('\.0', '', regex=True)
dfC['dateDeathA'] = dfC['dateDeathA'].astype(str).replace('\.0', '', regex=True)
dfC['dateFlA'] = dfC['dateFlA'].astype(str).replace('\.0', '', regex=True)
dfC['match_dateBirthB'] = dfC['match_dateBirthB'].astype(str).replace('\.0', '', regex=True)
dfC['match_dateDeathB'] = dfC['match_dateDeathB'].astype(str).replace('\.0', '', regex=True)
dfC['match_dateFlB'] = dfC['match_dateFlB'].astype(str).replace('\.0', '', regex=True)


In [ ]:
dfC.info()

#### Create joined / unique names and fill the blanks

In [ ]:
dfC['JoinedInitial'] = dfC['nameStringA'] + '^' + dfC['dateBirthA'] + '^' + dfC['dateDeathA'] + '^' + dfC['dateFlA']
dfC['JoinedMapped'] = dfC['match_nameStringB'] + '^' + dfC['match_dateBirthB']  + '^' + dfC['match_dateDeathB'] + '^' + dfC['match_dateFlB']

# Fill in blanks
dfC['JoinedMapped'] = dfC['JoinedMapped'].fillna('notmapped')

In [ ]:
dfC.head(5)

In [ ]:
# dfC.info()

#### Run the second script to detect variation in the mapped forms

In [ ]:
# Convert these joined names to strings
dfC['JoinedInitial'] = dfC['JoinedInitial'].astype('string')
dfC['JoinedMapped'] = dfC['JoinedMapped'].astype('string')

In [ ]:
dfC.info()

In [ ]:
# create a score to determine how much the initial and the map form of the name (including dates) varies
for j in dfC.index:
    clear_output(wait=True)
    rowIndex = dfC.index[j]
    initialForm = dfC.iloc[j,13]
    mappedForm = dfC.iloc[j,14]
    matchScoreFinal = fuzz.ratio(initialForm, mappedForm)
    print("Current progress loop1:", np.round(j/len(dfC) *100, 2),"%")
    if 0 <= matchScoreFinal <=100:
        dfC.loc[rowIndex, 'ScoreMappedVersionsNotChangedis100'] = matchScoreFinal        

In [ ]:
# dfC

In [ ]:
dfC.columns

In [ ]:
# Reorder the columns in a way that is easier to evaluate mapping

dfD = dfC[['JoinedInitial',
        'JoinedMapped',
        'personIdA',
        'match_personIdB',
        'scoreCase',
        'scoreType',
        'scoreNameString',
        'ScoreMappedVersionsNotChangedis100']]


In [ ]:
dfD.info()

In [ ]:
dfD.scoreCase.value_counts()

# Download the mapping candidates file

In [ ]:
# This file will contain the mapping candidates, which is easier to evaluate externally, e.g., in OpenRefine

# this is the local path to the processed data in your own computer where this file will be stored
pathProcDataFolder = f'/Users/Melga001/stack/workspace/SKILLNET-PRODUCTION/_sharedRepositoriesGithub/mappingPersonNames/data/processed/'

datasetA = 'ListA' #change list name if wanted
datasetB = 'ListB' #change list name if wanted
description = '' #add file description if wanted

#####bring back dfD
timestr = time.strftime("%Y%m%d-%H%M%S")
fileNameMappingCandidates = (f"{pathProcDataFolder}MappingsPersonsCandidates_{datasetA}vs{datasetB}_{description}_{timestr}.csv")
dfD.to_csv(fileNameMappingCandidates)